In [ ]:
import os
import math
import csv

import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import scipy.special as special
from scipy.stats import poisson, uniform, norm
import matplotlib.gridspec as gridspec
from tqdm import tqdm
from mpl_axes_aligner import align
from multiprocessing import Pool, cpu_count
from functools import partial
from scipy.signal import savgol_filter

import wf_func as wff

spe_pre = wff.read_model('spe.h5', 1)
p = spe_pre[0]['parameters']
gmu = 160.
gsigma = 40.
std = 1.

import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rcParams['font.size'] = 20
%matplotlib inline

# 使用单 bin 时间窗计算 $\mu$ 后验分布的 bias

In [ ]:
def getm(i, p):
    N = 10000
    bias = np.empty(N)
    np.random.seed(i)
    Mu = 10
    gmu = 160
    gsigma = 40
    npan = np.arange(1, 50)
    for i in range(N):
        n = poisson.ppf(1 - uniform.rvs(scale=1-poisson.cdf(0, Mu), size=1), Mu).astype(int)
        cha = norm.ppf(1 - uniform.rvs(scale=1-norm.cdf(0, loc=gmu, scale=gsigma), size=n), loc=gmu, scale=gsigma)
        cha = cha.sum()
        if p:
            weight = poisson.pmf(npan, Mu) * norm.pdf(cha, loc=gmu * npan, scale=gsigma * np.sqrt(npan))
        else:
            weight = norm.pdf(cha, loc=gmu * npan, scale=gsigma * np.sqrt(npan))
        weight = weight / weight.sum()
        bias[i] = np.sum(npan * weight) - n
    m = bias.mean()
    return m

In [ ]:
slices = np.arange(1000).T.tolist()
with Pool(100) as pool:
    resultpoisson = np.hstack([pool.starmap(getm, zip(slices, [True] * len(slices)))])
with Pool(100) as pool:
    resultflat = np.hstack([pool.starmap(getm, zip(slices, [False] * len(slices)))])

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(resultpoisson, bins=20, histtype='step', label='poisson')
ax.hist(resultflat, bins=20, histtype='step', label='flat')
ax.grid()
ax.legend(title='poisson mean = {:.2e}\nflat mean = {:.2e}'.format(resultpoisson.mean(), resultflat.mean()), loc='upper center')
ax.set_xlabel('bias')
ax.set_ylabel('count')
fig.savefig('note/onebinbias.png')
plt.show()

# 评估 DAQ 取整对 $\sigma_w$ 大小的影响

In [ ]:
window = 100
pan = np.arange(window)
charge = wff.charge(N, gmu=gmu, gsigma=gsigma, thres=0)
spetru = [wff.spe(pan, tau=p[0], sigma=p[1], A=p[2]) * charge[i] for i in range(N)]
spe = [np.around(spetru[i] + np.random.normal(0, std, size=window)) for i in range(N)]
diff = np.stack([spe[i] - spetru[i] for i in range(N)]).flatten()

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(-5, 5, 0.1)
ax.hist(diff, bins=200, alpha=0.5, histtype='step', density=True)
ax.plot(t, norm.pdf(t, loc=0, scale=std), c='k')
plt.show()

# $\mu$ 的 bias 随着 bin width 的变化

In [ ]:
with open('rc.csv') as f:
    f_csv = csv.reader(f, delimiter=' ')
    Tau = next(f_csv)
    Tau = [float(i) for i in Tau]
    Sigma = next(f_csv)
    Sigma = [float(i) for i in Sigma]

filelist = os.listdir('result/lucyddm/solu')
filelist = [f for f in filelist if f[0] != '.' and os.path.splitext(f)[-1] == '.h5']
numbers = [[float(i) for i in f[:-3].split('-')] for f in filelist]
stype = np.dtype([('mu', np.float64), ('tau', np.float64), ('sigma', np.float64), ('n', np.uint), ('stdmutru', np.float64), ('stdmuint', np.float64), ('stdmupe', np.float64), ('stdmumax', np.float64), ('stdmu', np.float64), ('biasmuint', np.float64), ('biasmupe', np.float64), ('biasmumax', np.float64), ('biasmu', np.float64), ('N', np.uint)])
mtsi = np.zeros(len(numbers), dtype=stype)
mtsi['mu'] = np.array([i[0] for i in numbers])
mtsi['tau'] = np.array([i[1] for i in numbers])
mtsi['sigma'] = np.array([i[2] for i in numbers])
mtsi['n'] = np.arange(len(numbers))
mtsi['stdmutru'] = np.nan
mtsi['stdmuint'] = np.nan
mtsi['stdmupe'] = np.nan
mtsi['stdmumax'] = np.nan
mtsi['stdmu'] = np.nan
mtsi['biasmuint'] = np.nan
mtsi['biasmupe'] = np.nan
mtsi['biasmumax'] = np.nan
mtsi['biasmu'] = np.nan
mtsi = np.sort(mtsi, kind='stable', order=['mu', 'tau', 'sigma'])
mts = {1:mtsi.copy(), 2:mtsi.copy(), 5:mtsi.copy(), 10:mtsi.copy()}

marker = {'int':'o', 'fbmp':'s', 'fbmpmax':'^'}
color = {'int':'g', 1:'r', 2:'b', 5:'y', 10:'m'}

for key in mts.keys():
    for i in range(len(mts[key])):
        f = filelist[mts[key][i]['n']]
        mu = mts[key][i]['mu']
        tau = mts[key][i]['tau']
        sigma = mts[key][i]['sigma']
        try:
            with h5py.File(os.path.join('result', 'fbmp-'+str(key), 'solu', f), 'r', libver='latest', swmr=True) as soluf, h5py.File(os.path.join('result', 'fbmp-'+str(key), 'dist', f), 'r', libver='latest', swmr=True) as distf, h5py.File(os.path.join('waveform', f), 'r', libver='latest', swmr=True) as wavef:
                time = soluf['starttime'][:]
                record = distf['Record'][:]
                start = wavef['SimTruth/T'][:]
                pelist = wavef['SimTriggerInfo/PEList'][:]
                waves = wavef['Readout/Waveform'][:]
                gmu = wavef['SimTriggerInfo/PEList'].attrs['gmu']
                gsigma = wavef['SimTriggerInfo/PEList'].attrs['gsigma']
                r = wavef['SimTruth/T'].attrs['r']
            mts[key][i]['N'] = len(start)
            vali = np.full(len(start['T0']), True)
            mts[key][i]['N'] = len(start)
            Chnum = len(np.unique(pelist['PMTId']))
            e_ans, i_ans = np.unique(pelist['TriggerNo'] * Chnum + pelist['PMTId'], return_index=True)
            i_ans = np.append(i_ans, len(pelist))
            pe_sum = np.array([pelist[i_ans[i]:i_ans[i+1]]['Charge'].sum() for i in range(len(e_ans))]) / gmu
            wave_sum = waves['Waveform'].sum(axis=1) / gmu
            n = np.arange(1, 1000)
            mean = np.average(n, weights=poisson.pmf(n, mu=mu))
            lognm = np.average(np.log(n), weights=poisson.pmf(n, mu=mu))
            slog = np.sqrt(np.average((np.log(n) - lognm)**2, weights=poisson.pmf(n, mu=mu)))
            mts[key][i]['stdmutru'] = slog
            slog = np.std(np.log(wave_sum[vali]), ddof=-1)
            m = np.mean(wave_sum[vali]) - mean
            mts[key][i]['stdmuint'] = slog
            mts[key][i]['biasmuint'] = m
            s = np.std(pe_sum[vali], ddof=-1)
            slog = np.std(np.log(pe_sum[vali]), ddof=-1)
            m = np.mean(pe_sum[vali]) - mean
            mts[key][i]['stdmupe'] = slog
            mts[key][i]['biasmupe'] = m
            s = np.std(time['mucharge'][vali], ddof=-1)
            slog = np.std(np.log(time['mucharge'][vali]), ddof=-1)
            m = np.mean(time['mucharge'][vali]) - mean
            mts[key][i]['stdmumax'] = slog
            mts[key][i]['biasmumax'] = m
            s = np.std(time['muwave'][vali], ddof=-1)
            slog = np.std(np.log(time['muwave'][vali]), ddof=-1)
            m = np.mean(time['muwave'][vali]) - mean
            mts[key][i]['stdmu'] = slog
            mts[key][i]['biasmu'] = m
        except:
            pass

figb = plt.figure(figsize=(len(Tau) * 5, len(Sigma) * 3))
gs = gridspec.GridSpec(1, 2, figure=figb, left=0.1, right=0.8, top=0.92, bottom=0.15, wspace=0.3, hspace=0.35)
for sigma, i in zip(Sigma, list(range(len(Sigma)))):
    for tau, j in zip(Tau, list(range(len(Tau)))):
        stdlistkey = mts[1][(mts[1]['tau'] == tau) & (mts[1]['sigma'] == sigma)]
        
        ax = figb.add_subplot(gs[i, j])
        n = np.arange(1, 1000)
        mean = np.array([np.average(n, weights=poisson.pmf(n, mu=mu)) for mu in stdlistkey['mu']])
        yerr = stdlistkey['biasmuint'] / np.sqrt(stdlistkey['N']) / mean
        ax.errorbar(stdlistkey['mu'], stdlistkey['biasmuint'] / mean, yerr=yerr, label='int', color=color['int'])
        
        for key in mts.keys():
            stdlistkey = mts[key][(mts[key]['tau'] == tau) & (mts[key]['sigma'] == sigma)]
            yerr = stdlistkey['biasmumax'] / np.sqrt(stdlistkey['N']) / mean
            ax.errorbar(stdlistkey['mu'], stdlistkey['biasmumax'] / mean, yerr=yerr, label='fbmpmax-'+str(key), color=color[key], marker=marker['fbmpmax'])
            yerr = stdlistkey['biasmu'] / np.sqrt(stdlistkey['N']) / mean
            ax.errorbar(stdlistkey['mu'], stdlistkey['biasmu'] / mean, yerr=yerr, label='fbmp-'+str(key), color=color[key], marker=marker['fbmp'])
        ax.set_xlabel(r'$N_{\mathrm{PE}}\ \mathrm{expectation}\ \mu$')
        ax.set_ylabel(r'$\frac{\Delta \mu}{\mu}$')
        ax.set_title(fr'$\tau={tau}\mathrm{{ns}},\,\sigma={sigma}\mathrm{{ns}}$')
        ax.yaxis.get_major_formatter().set_powerlimits((0, 0))
        ax.grid()
        if i == len(Sigma) - 1 and j == len(Tau) - 1:
            ax.legend(loc='upper left', bbox_to_anchor=(1., 0.9))
figb.savefig('note/vs-biasmu-bin.png')
plt.close(figb)

# 检查评估 FBMP 的先验分布对结果的影响

In [ ]:
para = '10.0-20-5'
Mu = 10
Tau = 20
Sigma = 5
wf = h5py.File('waveform/' + para + '.h5', 'r', libver='latest', swmr=True)
ch = h5py.File('result/fbmp/char/' + para + '.h5', 'r', libver='latest', swmr=True)
so = h5py.File('result/fbmp/solu/' + para + '.h5', 'r', libver='latest', swmr=True)
di = h5py.File('result/fbmp/dist/' + para + '.h5', 'r', libver='latest', swmr=True)

In [ ]:
truth = wf['SimTruth/T'][:]
charge = ch['photoelectron'][:]
answer = so['starttime'][:]
dist = di['Record'][:]

In [ ]:
n = np.arange(1, 1000)
mean = np.average(n, weights=poisson.pmf(n, mu=Mu))
s = np.sqrt(np.average((n - mean)**2, weights=poisson.pmf(n, mu=Mu)))

In [ ]:
plt.close()
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot()
ax.hist2d(np.abs(answer['tswave'] - truth['T0']), np.abs(answer['muwave'] - mean), bins=(100, 100))
# ax.legend()
ax.set_xlabel(r'$\Delta t_0/ns$')
ax.set_ylabel(r'$\Delta \mu$')
plt.show()

In [ ]:
np.corrcoef(np.abs(answer['tswave'] - truth['T0']), np.abs(answer['muwave'] - mean))

In [ ]:
plt.close()
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot()
xmin = min(answer['tscharge'] - truth['T0'])
xmax = max(answer['tscharge'] - truth['T0'])
for i in range(int(dist['NPE'].min()), int(dist['NPE'].max())):
    s = np.std((answer['tscharge'] - truth['T0'])[dist['NPE'] == i], ddof=-1)
    n = len((answer['tscharge'] - truth['T0'])[dist['NPE'] == i])
    ax.hist((answer['tscharge'] - truth['T0'])[dist['NPE'] == i], bins=np.arange(xmin, xmax), alpha=0.5, histtype='step', density=False, label=f'{i},{s:.02f},{n}')
ax.legend()
ax.set_yscale('log')
ax.set_xlabel(r'$\Delta t_0/ns$')
plt.show()

In [ ]:
plt.close()
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot()
xmin = min(charge['Charge'])
xmax = max(charge['Charge'])
t = np.arange(0, 1000, 0.1)
ax.plot(t, norm.pdf(t, loc=160., scale=40.) / (1 - norm.cdf(0, loc=160., scale=40.)), label='ChargePDF')
# for i in range(int(dist['NPE'].min()), int(dist['NPE'].max())):
for i in [2, 4, 6, 8, 10, 12]:
    triggerno = dist['TriggerNo'][dist['NPE'] == i]
    c = charge['Charge'][np.isin(charge['TriggerNo'], triggerno)]
    m = c.mean()
    n = len(c)
    ax.hist(c, bins=np.arange(xmin, xmax, 10), alpha=0.5, histtype='step', density=True, label=f'{i},{m:.02f},{n}')
ax.legend()
ax.set_xlim(xmin, xmax)
ax.set_xlabel(r'$Charge/mV·ns$')
plt.show()